# Entrenamiento Preliminar de Modelos

Este script realiza la búsqueda del mejor umbral de ciclos y modelo base para la predicción de roturas de membrana.

In [ ]:
# -*- coding: utf-8 -*-

import os
import warnings
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

# Importar configuración y utilidades compartidas
from config import (
    CSV_ENTRENAR, CSV_VALIDAR,
    RANDOM_STATE, UMBRALES_CICLOS, THRESHOLDS, MODELOS_DISPONIBLES, RESULTADOS_PRELIMINAR,
    PESO_FALSOS_POSITIVOS, configurar_logging
)
from utils import crear_derivadas, analizar_membranas, calcular_score_balanceado, evaluar_con_umbral

warnings.filterwarnings('ignore')

# Configurar logger
logger = configurar_logging(__name__)

MODELOS_ACTIVOS = MODELOS_DISPONIBLES

## Funciones de Modelos

In [3]:
# ================================================================================
# FUNCIONES DE MODELOS
# ================================================================================

def obtener_modelos():
    """
    Retorna un diccionario con los modelos configurados.
    
    Returns:
        Diccionario {nombre: instancia_modelo}
    """
    modelos = {
        'XGBoost': XGBClassifier(
            n_estimators=200, max_depth=5, learning_rate=0.05,
            scale_pos_weight=15, random_state=RANDOM_STATE, 
            eval_metric='aucpr', n_jobs=-1
        ),
        'LightGBM': LGBMClassifier(
            n_estimators=200, max_depth=5, learning_rate=0.05,
            class_weight='balanced', random_state=RANDOM_STATE, 
            verbose=-1, n_jobs=-1
        ),
        'RandomForest': RandomForestClassifier(
            n_estimators=200, max_depth=8,
            class_weight='balanced_subsample', 
            random_state=RANDOM_STATE, n_jobs=-1
        ),
        'ExtraTrees': ExtraTreesClassifier(
            n_estimators=200, max_depth=10,
            class_weight='balanced_subsample', 
            random_state=RANDOM_STATE, n_jobs=-1
        ),
        'HistGradientBoosting': HistGradientBoostingClassifier(
            max_iter=200, max_depth=5, learning_rate=0.05,
            class_weight='balanced', random_state=RANDOM_STATE
        ),
        'CatBoost': CatBoostClassifier(
            iterations=200, depth=5, learning_rate=0.05,
            auto_class_weights='Balanced', random_state=RANDOM_STATE, 
            verbose=0, allow_writing_files=False
        )
    }
    
    # Filtrar solo modelos activos
    return {k: v for k, v in modelos.items() if k in MODELOS_ACTIVOS}

## Carga de Datos

In [4]:
# ================================================================================
# CARGA DE DATOS
# ================================================================================

logger.info("Cargando datos...")

df_train = pd.read_csv(CSV_ENTRENAR)
df_test = pd.read_csv(CSV_VALIDAR)

# Eliminar columna de índice si existe
for df in [df_train, df_test]:
    if 'Indice' in df.columns:
        df.drop(columns=['Indice'], inplace=True)

logger.info(f"Datos cargados: {len(df_train)} train, {len(df_test)} test")

2026-02-19 13:00:15 - __main__ - INFO - Cargando datos...
2026-02-19 13:00:15 - __main__ - INFO - Datos cargados: 2055 train, 673 test


## Ingeniería de Características y Escalado

In [5]:
# ================================================================================
# INGENIERÍA DE CARACTERÍSTICAS
# ================================================================================

logger.info("Aplicando ingeniería de características...")

df_train_der = crear_derivadas(df_train)
df_test_der = crear_derivadas(df_test)

# Separar variables de objetivo
columnas = [c for c in df_train_der.columns if c != 'Ciclos']

X_train = df_train_der[columnas]
X_test = df_test_der[columnas]
y_train_ciclos = df_train['Ciclos'].values
y_test_ciclos = df_test['Ciclos'].values

# Escalado
logger.info("Escalando variables...")

scaler = StandardScaler()
X_train_esc = pd.DataFrame(scaler.fit_transform(X_train), columns=columnas)
X_test_esc = pd.DataFrame(scaler.transform(X_test), columns=columnas)

logger.info("Preprocesamiento completado.")

2026-02-19 13:00:18 - __main__ - INFO - Aplicando ingeniería de características...
2026-02-19 13:00:18 - __main__ - INFO - Escalando variables...
2026-02-19 13:00:18 - __main__ - INFO - Preprocesamiento completado.


## Búsqueda del Mejor Umbral de Ciclos

In [17]:
# ================================================================================
# BÚSQUEDA DEL MEJOR UMBRAL DE CICLOS
# ================================================================================

logger.info("Buscando mejor umbral de ciclos...")
logger.info("="*100)

mejores_por_umbral = {}
todos_resultados = []
modelos = obtener_modelos()

for umbral in UMBRALES_CICLOS:
    y_train_umbral = (y_train_ciclos < umbral).astype(int)
    y_test_umbral = (y_test_ciclos < umbral).astype(int)
    
    # Aplicar SMOTE
    k_neighbors = min(3, y_train_umbral.sum() - 1)
    smote = SMOTE(random_state=RANDOM_STATE, k_neighbors=k_neighbors)
    
    try:
        X_train_res, y_train_bal = smote.fit_resample(X_train_esc, y_train_umbral)
        X_train_bal = pd.DataFrame(X_train_res, columns=columnas)
    except ValueError as e:
        logger.warning(f"SMOTE falló para umbral {umbral}: {e}")
        X_train_bal = X_train_esc
        y_train_bal = y_train_umbral
    
    mejor_umbral = {'score': float('-inf')}
    
    for nombre, clf in modelos.items():
        clf.fit(X_train_bal, y_train_bal)
        y_proba = clf.predict_proba(X_test_esc)[:, 1]
        
        for th in THRESHOLDS:
            result = evaluar_con_umbral(y_test_umbral, y_proba, y_test_ciclos, umbral, th)
            if result:
                score = calcular_score_balanceado(
                    result['pct_membranas'],
                    result['FP'],
                    result['membranas_total'],
                    PESO_FALSOS_POSITIVOS
                )

                todos_resultados.append({
                    'umbral_ciclos':        umbral,
                    'modelo':               nombre,
                    'score_balanceado':     score,
                    'threshold':            th,
                    'FP':                   result['FP'],
                    'membranas_detectadas': result['membranas_detectadas'],
                    'membranas_total':      result['membranas_total'],
                    'TP':                   result['TP'],
                    'FN':                   result['FN'],
                    'TN':                   result['TN'],
                })

                if score > mejor_umbral['score']:
                    mejor_umbral = {
                        'umbral': umbral,
                        'modelo': nombre,
                        'threshold': th,
                        'score': score,
                        'pct_membranas': result['pct_membranas'],
                        'fp': result['FP'],
                        'resultado': result
                    }
    
    if mejor_umbral['score'] > float('-inf'):
        mejores_por_umbral[umbral] = mejor_umbral
        logger.info(f"MEJOR PARA UMBRAL {umbral}: {mejor_umbral['modelo']} (Score={mejor_umbral['score']:.2f})")


# ================================================================================
# GUARDAR RESULTADOS EN CSV
# ================================================================================

if todos_resultados:
    df_resultados = pd.DataFrame(todos_resultados)
    df_resultados = df_resultados.sort_values(by='score_balanceado', ascending=False)

    # Seleccionar columnas en el orden definido
    columnas_csv = ['umbral_ciclos', 'modelo', 'threshold', 'score_balanceado', 'membranas_total', 'membranas_detectadas', 'TP', 'FP', 'FN', 'TN']
    df_resultados[columnas_csv].to_csv(RESULTADOS_PRELIMINAR, index=False)
    logger.info("="*100)
    logger.info(f"{len(todos_resultados)} resultados guardados en: {RESULTADOS_PRELIMINAR}")
else:
    logger.warning("No se generaron resultados válidos para guardar.")


2026-02-19 13:53:39 - __main__ - INFO - Buscando mejor umbral de ciclos...
2026-02-19 13:53:39 - __main__ - INFO - ====================================================================================================
2026-02-19 13:53:41 - __main__ - INFO - MEJOR PARA UMBRAL 3: HistGradientBoosting (Score=56.00)
2026-02-19 13:53:43 - __main__ - INFO - MEJOR PARA UMBRAL 5: ExtraTrees (Score=48.00)
2026-02-19 13:53:45 - __main__ - INFO - MEJOR PARA UMBRAL 7: LightGBM (Score=56.00)
2026-02-19 13:53:47 - __main__ - INFO - MEJOR PARA UMBRAL 9: LightGBM (Score=60.00)
2026-02-19 13:53:47 - __main__ - INFO - ====================================================================================================
2026-02-19 13:53:47 - __main__ - INFO - 456 resultados guardados en: /Users/daniel/Desktop/TFG/TFG/output/resultados_preliminar.csv
